In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

import re
import pandas as pd
import time 
from typing import *
from tqdm import tqdm
import os
import datetime
import tkinter as tk
from tkinter import ttk, messagebox
import threading

# Meriam scraper

All words are scraped from https://www.merriam-webster.com

In [ ]:
class scraper:
    def __init__(self, timeout: int = 2):

        self.timeout = timeout 

        userAgent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
        options = Options()
        options.add_argument(f'user-agent={userAgent}')
        self.driver = webdriver.Chrome(options=options)
        self.driver.get("https://www.merriam-webster.com/wordfinder/classic/contains/all/")
    
    def get_words_one_page(self) -> List[str]:
        words = []
        word_list = self.driver.find_element(By.XPATH, './/div[@class="panel-body"]')
        word_elements = word_list.find_elements(By.XPATH, './/li[@class="pb-4 d-flex"]')
        for word_element in word_elements:
            words.append(word_element.text.strip())
        
        return words
    
    def wrangle

In [ ]:
userAgent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
options = Options()
options.add_argument(f'user-agent={userAgent}')
driver = webdriver.Chrome(options=options)
driver.get("https://www.merriam-webster.com/wordfinder/classic/contains/all/")

In [ ]:
word_list = driver.find_element(By.XPATH, './/div[@class="panel-body"]')
word_elements = word_list.find_elements(By.XPATH, './/li[@class="pb-4 d-flex"]')
for i in tqdm(range(len(word_elements)), desc="Extracting words"):
    word = word_elements[i].text.strip()



In [ ]:
for i in tqdm(range(len(word_elements)), desc="Extracting words"):
    word = word_elements[i].text.strip()
    print(word)

Extracting words:   1%|          | 5/468 [00:00<00:10, 43.37it/s]

AAAs https://www.merriam-webster.com/dictionary/AAAs
aahs https://www.merriam-webster.com/dictionary/aahs
aayn https://www.merriam-webster.com/dictionary/aayn
abas https://www.merriam-webster.com/dictionary/abas
abba https://www.merriam-webster.com/dictionary/abba
abbe https://www.merriam-webster.com/dictionary/abbe
abbs https://www.merriam-webster.com/dictionary/abbs
ABCs https://www.merriam-webster.com/dictionary/ABCs
ABDs https://www.merriam-webster.com/dictionary/ABDs


Extracting words:   3%|▎         | 15/468 [00:00<00:11, 39.33it/s]

abed https://www.merriam-webster.com/dictionary/abed
Abel https://www.merriam-webster.com/dictionary/Abel
abet https://www.merriam-webster.com/dictionary/abet
Abib https://www.merriam-webster.com/dictionary/Abib
abir https://www.merriam-webster.com/dictionary/abir
able https://www.merriam-webster.com/dictionary/able
ably https://www.merriam-webster.com/dictionary/ably
ABMs https://www.merriam-webster.com/dictionary/ABMs
Abor https://www.merriam-webster.com/dictionary/Abor
abox https://www.merriam-webster.com/dictionary/abox


Extracting words:   5%|▌         | 25/468 [00:00<00:10, 41.58it/s]

...

bean https://www.merriam-webster.com/dictionary/bean
bear https://www.merriam-webster.com/dictionary/bear
beat https://www.merriam-webster.com/dictionary/beat
beau https://www.merriam-webster.com/dictionary/beau
Beja https://www.merriam-webster.com/dictionary/Beja
beka https://www.merriam-webster.com/dictionary/beka
bema https://www.merriam-webster.com/dictionary/bema
Bena https://www.merriam-webster.com/dictionary/Bena


Extracting words:  99%|█████████▉| 464/468 [00:11<00:00, 38.02it/s]

besa https://www.merriam-webster.com/dictionary/besa
beta https://www.merriam-webster.com/dictionary/beta
bhai https://www.merriam-webster.com/dictionary/bhai
Bhar https://www.merriam-webster.com/dictionary/Bhar
BHAs https://www.merriam-webster.com/dictionary/BHAs
bhat https://www.merriam-webster.com/dictionary/bhat
bias https://www.merriam-webster.com/dictionary/bias


Extracting words: 100%|██████████| 468/468 [00:11<00:00, 40.68it/s]

biga https://www.merriam-webster.com/dictionary/biga
bija https://www.merriam-webster.com/dictionary/bija
